# Evaluation MetaCAT - BiLSTM

In [1]:
from tokenizers import ByteLevelBPETokenizer
import os

from medcat.cat import CAT
from medcat.vocab import Vocab
from medcat.cdb import CDB
from medcat.config import Config
from medcat.meta_cat import MetaCAT

In [2]:
# Input
data_dir = os.path.join('..', 'data')
cdb_file = os.path.join(data_dir, 'cdb.dat')
vocab_file = os.path.join(data_dir, 'vocab.dat')

# Output
output_dir = 'output'

# Name should contain 'bbpe' for ByteLevelBPETokenizer or 'bert' for BertTokenizerFast
tokenizer_name = 'bbpe_dutch-wikipedia'

## Load tokenizer

In [3]:
# Create, train and save the tokenizer
mc_negation = MetaCAT()
mc_negation = mc_negation.load(save_dir=output_dir, add_prefix_space=True)

## Example usage

In [4]:
# Load the cdb and vocab 
config = Config()

vocab = Vocab.load(vocab_file)
cdb = CDB.load(cdb_file)

# Create MedCAT pipeline
cat = CAT(cdb=cdb, vocab=vocab, config=config, meta_cats=[mc_negation])

In [5]:
# Test on DL1114 from DCC with negation
text = 'Echo- en rontgenonderzoek van de heup toont geen evidente heupdysplasie.'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

Entity: heup
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.9724474, 'name': 'Negation'}}


Entity: heupdysplasie
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.9999876, 'name': 'Negation'}}




In [6]:
# Test on DL1114 from DCC without negation
text = 'Echo- en rontgenonderzoek van de heup toont evidente heupdysplasie.'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

Entity: heup
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.9985293, 'name': 'Negation'}}


Entity: heupdysplasie
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.99687064, 'name': 'Negation'}}




## Additional testing

In [7]:
# Test on DL1112 from DCC
text = 'Patient kan zich geen trauma herinneren.'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

# Trauma is not identified as medical concept.

In [8]:
# Test on NTvG article
# https://www.ntvg.nl/artikelen/acute-buik-op-basis-van-een-wandelende-milt
text = '1 maand na de operatie had patiënte geen buikpijn meer en was zij goed hersteld.'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")
    
# The negation was correctly identified.

Entity: operatie
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.99955237, 'name': 'Negation'}}


Entity: buikpijn
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.72627723, 'name': 'Negation'}}




In [9]:
# Test on NTvG article
# https://www.ntvg.nl/artikelen/een-bezoar-bij-een-vrouw-met-clomipramine-intoxicatie
text = 'Er waren geen tekenen van darmobstructie te zien.'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

# Correct identification of negation, but incorrect linking of zien

Entity: darmobstructie
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.98679256, 'name': 'Negation'}}


Entity: zien
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.9872715, 'name': 'Negation'}}




In [10]:
# Test on NTvG article
# https://www.ntvg.nl/artikelen/nieuws/vaker-ziek-na-acute-fase-covid-19
text = 'Alle patiënten werden gematcht met een controlegroep bij wie geen SARS-CoV-2-infectie was geregistreerd.'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")
    
# Correct identification of negation

Entity: patiënten
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.9977733, 'name': 'Negation'}}


Entity: controlegroep
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.99956954, 'name': 'Negation'}}


Entity: SARS-CoV-2-infectie
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.845927, 'name': 'Negation'}}




In [11]:
text = 'Er zijn geen bijwerkingen gemeld van de scan'
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")
# Negation was incorrectly identified
# Entity linking of bijwerkingen was missed

Entity: scan
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.8478781, 'name': 'Negation'}}




### Check add_prefix_space

In [12]:
text = " Zwelling treedt niet op."
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

# Negation was not identified. But difficult one, see next.

Entity: Zwelling
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.967743, 'name': 'Negation'}}




In [13]:
text = "Zwelling treedt niet op."
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

# Negation was not identified. But difficult one, see next.

Entity: Zwelling
Meta Annotations: {'Negation': {'value': 'not negated', 'confidence': 0.967743, 'name': 'Negation'}}




In [14]:
text = "Geen zwelling treedt op."
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

# Negation was identified

Entity: zwelling
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.99528426, 'name': 'Negation'}}




In [15]:
text = "Geen zwelling treedt niet op."
doc = cat(text)
for ent in doc.ents:
    print("Entity: " + ent.text)
    print("Meta Annotations: " + str(ent._.meta_anns))
    print("\n")

# Negation was identified

Entity: zwelling
Meta Annotations: {'Negation': {'value': 'negated', 'confidence': 0.9988198, 'name': 'Negation'}}




## Evaluate MetaCat on subsets of the data
The ContextD paper calculates precision, recall and F1-score on subsets of the data. In this section we calculate the same scores with the just created model. Note that this results in a calculation on a set of data that was included during the training phase. For proper score calculations, we will do cross validation at a later stage.

In [16]:
json_file_DL = os.path.join(data_dir, 'emc-dcc_ann_DL.json')
json_file_GP = os.path.join(data_dir, 'emc-dcc_ann_GP.json')
json_file_RD = os.path.join(data_dir, 'emc-dcc_ann_RD.json')
json_file_SP = os.path.join(data_dir, 'emc-dcc_ann_SP.json')

In [17]:
mc_negation.eval(json_file_RD)

**************************************************  Test
              precision    recall  f1-score   support

           0       0.96      0.90      0.93       595
           1       0.98      0.99      0.99      3088

    accuracy                           0.98      3683
   macro avg       0.97      0.95      0.96      3683
weighted avg       0.98      0.98      0.98      3683

Test Loss:  0.0826881697678284








{'0': {'precision': 0.9605734767025089,
  'recall': 0.9008403361344538,
  'f1-score': 0.9297484822202948,
  'support': 595},
 '1': {'precision': 0.98112,
  'recall': 0.9928756476683938,
  'f1-score': 0.9869628198937711,
  'support': 3088},
 'accuracy': 0.9780070594623947,
 'macro avg': {'precision': 0.9708467383512545,
  'recall': 0.9468579919014237,
  'f1-score': 0.958355651057033,
  'support': 3683},
 'weighted avg': {'precision': 0.9778006458425177,
  'recall': 0.9780070594623947,
  'f1-score': 0.9777196673236602,
  'support': 3683}}

In [18]:
mc_negation.eval(json_file_SP)

**************************************************  Test
              precision    recall  f1-score   support

           0       0.95      0.74      0.83       416
           1       0.95      0.99      0.97      2309

    accuracy                           0.95      2725
   macro avg       0.95      0.86      0.90      2725
weighted avg       0.95      0.95      0.95      2725

Test Loss:  0.15795741708383762








{'0': {'precision': 0.9503105590062112,
  'recall': 0.7355769230769231,
  'f1-score': 0.8292682926829268,
  'support': 416},
 '1': {'precision': 0.9542238868081565,
  'recall': 0.9930705933304461,
  'f1-score': 0.9732597623089984,
  'support': 2309},
 'accuracy': 0.9537614678899082,
 'macro avg': {'precision': 0.9522672229071838,
  'recall': 0.8643237582036847,
  'f1-score': 0.9012640274959626,
  'support': 2725},
 'weighted avg': {'precision': 0.9536264760317861,
  'recall': 0.9537614678899082,
  'f1-score': 0.9512779452945229,
  'support': 2725}}

In [19]:
mc_negation.eval(json_file_DL)

**************************************************  Test
              precision    recall  f1-score   support

           0       0.98      0.90      0.94       379
           1       0.98      1.00      0.99      2417

    accuracy                           0.98      2796
   macro avg       0.98      0.95      0.96      2796
weighted avg       0.98      0.98      0.98      2796

Test Loss:  0.059427651310605664








{'0': {'precision': 0.9798850574712644,
  'recall': 0.899736147757256,
  'f1-score': 0.938101788170564,
  'support': 379},
 '1': {'precision': 0.9844771241830066,
  'recall': 0.9971038477451386,
  'f1-score': 0.9907502569373073,
  'support': 2417},
 'accuracy': 0.9839055793991416,
 'macro avg': {'precision': 0.9821810908271356,
  'recall': 0.9484199977511973,
  'f1-score': 0.9644260225539356,
  'support': 2796},
 'weighted avg': {'precision': 0.9838546659270158,
  'recall': 0.9839055793991416,
  'f1-score': 0.9836137155701415,
  'support': 2796}}

In [20]:
mc_negation.eval(json_file_GP)

**************************************************  Test
              precision    recall  f1-score   support

           0       0.95      0.71      0.81       383
           1       0.96      1.00      0.98      3024

    accuracy                           0.96      3407
   macro avg       0.96      0.85      0.90      3407
weighted avg       0.96      0.96      0.96      3407

Test Loss:  0.1171220576257578








{'0': {'precision': 0.951048951048951,
  'recall': 0.7101827676240209,
  'f1-score': 0.8131539611360239,
  'support': 383},
 '1': {'precision': 0.9644344761294457,
  'recall': 0.9953703703703703,
  'f1-score': 0.9796582587469487,
  'support': 3024},
 'accuracy': 0.9633108306427942,
 'macro avg': {'precision': 0.9577417135891984,
  'recall': 0.8527765689971956,
  'f1-score': 0.8964061099414863,
  'support': 3407},
 'weighted avg': {'precision': 0.9629297340966222,
  'recall': 0.9633108306427942,
  'f1-score': 0.9609405757457793,
  'support': 3407}}